# DEV END

### Importamos librerias

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def main(url):
    links_secciones = _obtener_secciones(url)
    notas = _obtener_notas(links_secciones)
    data = _obtener_data(notas)
    _save_data(data)

In [3]:
def _obtener_secciones(url):
    request = requests.get(url)
    
    if request.status_code == 200:
        soup = BeautifulSoup(request.text, 'html.parser')
        secciones = soup.find('div', attrs= {'class':'moduletable_modmenulateral'}).find_all('li')
        links = [seccion.a.get('href') for seccion in secciones]
        links_secciones = [url + i  for i in links ]
    return links_secciones


In [4]:
# _obtener_secciones('https://www.mef.gob.pe')

In [5]:
def _obtener_notas(links_secciones):
    notas = []
    for link in links_secciones:
        try:
            r = requests.get(link)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                notas.extend(__obtener_urls_notas(soup))
            else:
                print('No se pudo obtener la seccion', link)
        except:
            print('No se pudo obtener la seccion', link)

    return notas

In [6]:

def __obtener_urls_notas(soup):
    '''
    Funcion que recibe un objeto de BeautifulSoup de una pagina 
    de una seccion y devuelve una lista de urls a las notas de esa seccion.
    '''
    lista_notas = []

    # Obtengo el articulo promocionado
    featured_article = soup.find(
        'div', attrs={'class': 'featured-article__container'})
    if featured_article:
        lista_notas.append(featured_article.a.get('href'))

    # Obtengo el listado de los articulos
    article_list = soup.find('ul', attrs={'class': 'article-list'})

    for article in article_list.find_all('li'):

        if article.a:
            lista_notas.append(article.a.get('href'))

    return list(set(lista_notas))


In [7]:
def _obtener_data(notas):
    data = []

    for i, nota in enumerate(notas):
        print( {i}/{len(notas)})
        data.append(__scrape_nota(nota))
    return data


In [8]:
def __scrape_nota(url):
    try:
        nota = requests.get(url)
    except Excepton as e:
        print('Error scrapeando ULR' ,{url})
        print(e)
        returnNone

    if nota.status_code != 200:
        print('fError obteniendo nota', {url})
        print('Status code ', {nota.status_code})
        returnNone
        
    s_nota = BeautifulSoup(nota.text, 'html.parser')

    ret_dict = __obtener_info(s_nota)
    ret_dict['url'] = url

    return ret_dict        

In [9]:

def __obtener_info(s_nota):
    # Creamos un diccionario vacio para probarlo con la informacion
    ret_dict = {}

    # Extraemos la fecha
    fecha = s_nota.find('span', attrs={'pubdate': 'pubdate'})
    if fecha:
        ret_dict['fecha'] = fecha.get('datetime')
    else:
        ret_dict['fecha'] = None

    # Extraemos el titulo
    titulo = s_nota.find('h1', attrs={'class': 'article-title'})
    if titulo:
        ret_dict['titulo'] = titulo.text
    else:
        ret_dict['titulo'] = None

    # Extraer la volanda
    volanta = s_nota.find('h2', attrs={'class': 'article-prefix'})
    if volanta:
        ret_dict['volanta'] = volanta.get_text()
    else:
        ret_dict['volanta'] = None

    # Extraer copete
    copete = s_nota.find('div', attrs={'class': 'article-summary'})
    if copete:
        ret_dict['copete'] = copete.get_text()
    else:
        ret_dict['copete'] = None

    # Extraer autor
    autor = s_nota.find('div', attrs={'class': 'article-author'})
    if autor:
        ret_dict['autor'] = autor.get_text()
    else:
        ret_dict['autor'] = None

    # Extraer imagen
    media = s_nota.find('div', attrs={'class': 'article-main-media'})
    if media:
        imagenes = media.find_all('img')
        if len(imagenes) == 0:
            print('no se encontraron imagenes')
        else:
            imagen = imagenes[-1]
            img_src = imagen.get('data-src')
            try:
                img_req = requests.get(img_src)
                if img_req.status_code == 200:
                    ret_dict['imagen'] = img_req.content
                else:
                    ret_dict['imagen'] = None
            except:
                print('No se pudo obtener la imagen')
    else:
        print('No se encontro media')

    # Extraerel  cuerpo
    cuerpo = s_nota.find('div', attrs={'class': 'article-text'})
    if cuerpo:
        ret_dict['cuerpo'] = cuerpo.get_text()
    else:
        ret_dict['cuerpo'] = None

    return ret_dict


In [10]:
def _save_data(data):
    df = pd.DataFrame(data)
    df.to_csv('Notas_pagina12.csv')
        
    return df


In [11]:
if __name__ == "__main__":
    url = 'https://www.mef.gob.pe/es/presentacion'
    main(url)
    

('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/presentacion')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/proyecciones-macroeconomicas')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/seguimiento-de-la-ejecucion-presupuestal-consulta-amigable')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/seguimiento-a-las-acciones-prioritarias')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/transferencias-a-gobierno-nacional-regional-y-locales/que-es-y-como-muestra-la-consulta')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/recaudacion')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/fondos-del-estado')
('No se pudo obtener la seccion', u'https://www.mef.gob.pe/es/presentacion/es/provedores-del-estado/que-es-y-como-muestra-la-consulta')
('No se pudo obtener la seccion', u'https://w

NameError: global name 'pd' is not defined

# START HERE


In [12]:
import requests

In [13]:
url = 'https://www.mef.gob.pe/es/presentacion'


In [14]:
text = requests.get(url)

In [15]:
text.status_code ###Codigo status http

200

In [16]:
print(text.text)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"> <html xmlns="http://www.w3.org/1999/xhtml" xml:lang="es-es" lang="es-es" dir="ltr" > <head> <meta http-equiv='X-UA-Compatible' content='IE=8'> <base href="https://www.mef.gob.pe/es/presentacion" /> <meta http-equiv="content-type" content="text/html; charset=utf-8" /> <meta name="description" content="Ministerio de Economía y Finanzas" /> <meta name="generator" content="Joomla! - Open Source Content Management" /> <title>Presentación</title> <link href="/es/presentacion" rel="canonical" /> <link href="/templates/meftemplaten2.3bannerssubsites/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon" /> <link rel="stylesheet" href="/plugins/content/xtypo/themes/default/style.css" type="text/css" /> <style type="text/css"> div.mod_search10083 input[type="search"]{ width:auto; } </style> <script src="/plugins/content/xtypo/assets/script.js" type="text/javascript"

In [17]:
text.content

'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"> <html xmlns="http://www.w3.org/1999/xhtml" xml:lang="es-es" lang="es-es" dir="ltr" > <head> <meta http-equiv=\'X-UA-Compatible\' content=\'IE=8\'> <base href="https://www.mef.gob.pe/es/presentacion" /> <meta http-equiv="content-type" content="text/html; charset=utf-8" /> <meta name="description" content="Ministerio de Econom\xc3\xada y Finanzas" /> <meta name="generator" content="Joomla! - Open Source Content Management" /> <title>Presentaci\xc3\xb3n</title> <link href="/es/presentacion" rel="canonical" /> <link href="/templates/meftemplaten2.3bannerssubsites/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon" /> <link rel="stylesheet" href="/plugins/content/xtypo/themes/default/style.css" type="text/css" /> <style type="text/css"> div.mod_search10083 input[type="search"]{ width:auto; } </style> <script src="/plugins/content/xtypo/assets/script.js" typ

In [18]:
text.headers

{'Transfer-Encoding': 'chunked', 'Set-Cookie': 'd13fb52848428fc6e057d1f488a5480b=m5ab2njkgksopki78cp1qo2so3; path=/; HttpOnly, 6a6e4b481ae3eb1f044fb4d0fac4d230=es-ES; path=/; secure, f5avrbbbbbbbbbbbbbbbb=HFEJFPAPHDJMCGGEKEKIAIFEGHKHNILOKDINNLLNOCIDOEJDLBGNDENKHJBFPGILGMAECBDJAMADCMLMMOGJBLOHBIEABKNMBPKCHEJCHDAFHJCIIHKKHAKFBMGBCOIO; HttpOnly; secure, f5_cspm=1234;, TS010d8927=01c1c6ff7a439a400ee543f183c153fb57ac794efbeb340a026ab23a49f52b788d624215e75213965beee2fee7f08cd20261ee2af4; Path=/', 'Expires': 'Wed, 17 Aug 2005 00:00:00 GMT', 'Keep-Alive': 'timeout=15, max=100', 'Last-Modified': 'Mon, 06 Jul 2020 19:47:21 GMT', 'Connection': 'Keep-Alive', 'Pragma': 'no-cache', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Date': 'Mon, 06 Jul 2020 19:47:20 GMT', 'P3P': 'CP="NOI ADM DEV PSAi COM NAV OUR OTRo STP IND DEM"', 'Content-Type': 'text/html; charset=utf-8'}

In [19]:
text.request.headers

{'Connection': 'keep-alive', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'User-Agent': 'python-requests/2.20.0'}

In [20]:
text.request.method

'GET'

In [21]:
text.request.url

'https://www.mef.gob.pe/es/presentacion'

In [22]:
from bs4 import BeautifulSoup

In [23]:
t= BeautifulSoup(text.text,"lxml")

In [24]:
type(t)

bs4.BeautifulSoup

In [25]:
print(t.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html dir="ltr" lang="es-es" xml:lang="es-es" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="IE=8" http-equiv="X-UA-Compatible"/>
  <base href="https://www.mef.gob.pe/es/presentacion"/>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="Ministerio de Economía y Finanzas" name="description"/>
  <meta content="Joomla! - Open Source Content Management" name="generator"/>
  <title>
   Presentación
  </title>
  <link href="/es/presentacion" rel="canonical"/>
  <link href="/templates/meftemplaten2.3bannerssubsites/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
  <link href="/plugins/content/xtypo/themes/default/style.css" rel="stylesheet" type="text/css"/>
  <style type="text/css">
   div.mod_search10083 input[type="search"]{ width:auto; }
  </style>
  <script src="/plugins/content/xtypo/assets/scrip

In [26]:
secciones = t.find('div', attrs= {'class':'moduletable_modmenulateral'}).find_all('li')

In [27]:
seccion = secciones[0]

In [28]:
seccion

<li class="item-100931 current active"><a href="/es/presentacion">Presentaci\xf3n</a></li>

In [29]:
seccion.find('a')

<a href="/es/presentacion">Presentaci\xf3n</a>

In [30]:
seccion.a

<a href="/es/presentacion">Presentaci\xf3n</a>

In [31]:
seccion.a.get_text()

u'Presentaci\xf3n'

In [32]:
url_main = 'https://www.mef.gob.pe'

In [33]:
link = seccion.a.get('href')
link

'/es/presentacion'

In [34]:
seccion.a.get('href')

'/es/presentacion'

In [35]:
links_secciones = [seccion.a.get('href') for seccion in secciones]

In [36]:
links_secciones

['/es/presentacion',
 '/es/proyecciones-macroeconomicas',
 '/es/seguimiento-de-la-ejecucion-presupuestal-consulta-amigable',
 '/es/seguimiento-a-las-acciones-prioritarias',
 '/es/transferencias-a-gobierno-nacional-regional-y-locales/que-es-y-como-muestra-la-consulta',
 '/es/recaudacion',
 '/es/fondos-del-estado',
 '/es/provedores-del-estado/que-es-y-como-muestra-la-consulta',
 '/es/consulta-de-deuda-publica',
 '/es/cuenta-general-de-la-republica-sp-26800/que-es-y-como-muestra-la-consulta',
 '/es/ejecucion-de-proyectos-de-inversion',
 '/es/fonafe',
 '/es/preguntas-frecuentes-sp-24472',
 '/es/normas-legales',
 '/es/manual-de-uso',
 '/es/documentacion-sp-3092/leyes-de-presupuesto-del-sector-publico']

In [37]:
t = ["https://www.mef.gob.pe" + i  for i in links_secciones ]

In [38]:
t

['https://www.mef.gob.pe/es/presentacion',
 'https://www.mef.gob.pe/es/proyecciones-macroeconomicas',
 'https://www.mef.gob.pe/es/seguimiento-de-la-ejecucion-presupuestal-consulta-amigable',
 'https://www.mef.gob.pe/es/seguimiento-a-las-acciones-prioritarias',
 'https://www.mef.gob.pe/es/transferencias-a-gobierno-nacional-regional-y-locales/que-es-y-como-muestra-la-consulta',
 'https://www.mef.gob.pe/es/recaudacion',
 'https://www.mef.gob.pe/es/fondos-del-estado',
 'https://www.mef.gob.pe/es/provedores-del-estado/que-es-y-como-muestra-la-consulta',
 'https://www.mef.gob.pe/es/consulta-de-deuda-publica',
 'https://www.mef.gob.pe/es/cuenta-general-de-la-republica-sp-26800/que-es-y-como-muestra-la-consulta',
 'https://www.mef.gob.pe/es/ejecucion-de-proyectos-de-inversion',
 'https://www.mef.gob.pe/es/fonafe',
 'https://www.mef.gob.pe/es/preguntas-frecuentes-sp-24472',
 'https://www.mef.gob.pe/es/normas-legales',
 'https://www.mef.gob.pe/es/manual-de-uso',
 'https://www.mef.gob.pe/es/docum

In [39]:
sec = requests.get(t[0])

In [40]:
sec.status_code

200

In [41]:
s_seccion = BeautifulSoup(sec.text, 'lxml')

In [42]:
print(s_seccion)

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html dir="ltr" lang="es-es" xml:lang="es-es" xmlns="http://www.w3.org/1999/xhtml"> <head> <meta content="IE=8" http-equiv="X-UA-Compatible"/> <base href="https://www.mef.gob.pe/es/presentacion"/> <meta content="text/html; charset=utf-8" http-equiv="content-type"/> <meta content="Ministerio de Economía y Finanzas" name="description"/> <meta content="Joomla! - Open Source Content Management" name="generator"/> <title>Presentación</title> <link href="/es/presentacion" rel="canonical"/> <link href="/templates/meftemplaten2.3bannerssubsites/favicon.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/> <link href="/plugins/content/xtypo/themes/default/style.css" rel="stylesheet" type="text/css"/> <style type="text/css"> div.mod_search10083 input[type="search"]{ width:auto; } </style> <script src="/plugins/content/xtypo/assets/script.js" type="text/javascript"></scri

In [43]:
item_seccion = s_seccion.find('div', attrs={"class":"item-page"})

In [44]:
print(item_seccion)

<div class="item-page"> <h1> Presentación </h1> <h2 class="contentheading"> ¿Qué es el Portal de Transparencia Económica? </h2> <p> </p> <p style="text-align: justify;"><span class="texto_contenidos">El Portal de Transparencia Económica es una plataforma informativa de acceso libre que permite a cualquier usuario disponer, en tiempo real, de la más completa información económica que haya podido centralizar el Ministerio de Economía y Finanzas (MEF), desde el Gobierno de Transición a la fecha.<br/><br/></span> </p> <p style="text-align: justify;"><span class="texto_contenidos"><br/><br/>Está compuesto de Módulos de Consulta a Base de Datos con información financiera y presupuestal.</span> </p> <p style="text-align: justify;"><span class="texto_contenidos">El Portal se inauguró oficialmente el 23 de febrero del 2001, siendo Ministro de Economía y Finanzas, el Dr. Javier Silva Ruete, cuando se entregó al presidente de la República, Dr. Valentín Paniagua Corazao, el CD-ROM del Portal en un

In [45]:
item_seccion.a.get('href')

'/es/presentacion#top'

In [46]:
list_seccion = s_seccion.find('span', attrs={"class":"texto_linkeables"} )

In [47]:
list_seccion

<span class="texto_linkeables"><a href="/es/presentacion#top">subir</a></span>

In [48]:
list_seccion.a.get('href')

'/es/presentacion#top'

In [ ]:
#article_list = s_seccion.find('ul', attrs={"class":"item-page"})

In [ ]:
#article_list

In [ ]:
# import boto3
# import json
# import os
# import threading
# import sys
# from boto3.s3.transfer import TransferConfig

#### Definiendo variables

In [ ]:
# BUCKET_NAME = 'tem-um-bom-2'

#### Client y Resource

In [ ]:
# s3_client = boto3.client('s3')

In [ ]:
# s3_resource = boto3.resource('s3')

#### Gestion de buckets

In [ ]:
# s3_client.create_bucket(
#         Bucket=BUCKET_NAME,
#         CreateBucketConfiguration={
#             'LocationConstraint': 'eu-central-1'
#         }
#     )

In [ ]:
# s3_client.list_buckets()

In [ ]:
# s3_client.delete_bucket(Bucket=BUCKET_NAME)

In [ ]:
# object_key = 'File1.txt'
# s3_client.get_object(Bucket=BUCKET_NAME, Key=object_key)

In [ ]:
# file_path = 'michael-martin.pdf'
# s3_client.upload_file(file_path, BUCKET_NAME, 'michael-martin.pdf')

In [ ]:
# config = TransferConfig(multipart_threshold=1024 * 25, max_concurrency=10,
#                         multipart_chunksize=1024 * 25, use_threads=True)
# file_path = 'michael-martin.pdf'
# key_path = 'multipart_files/largefile.pdf'
# s3_resource.meta.client.upload_file(file_path, BUCKET_NAME, key_path,
#                                       ExtraArgs={'ACL': 'public-read', 'ContentType': 'text/pdf'},
#                                       Config=config
#                                       )

#### Session

In [ ]:
# aws_manager = boto3.session.Session(profile_name='administrador')

In [ ]:
# s3client_from_session = aws_manager.client('s3')

In [ ]:
# s3client_from_session.list_buckets()

#### Meta

In [ ]:
# s3_resource.meta.client## de resource a client

In [ ]:
# aws configure --profile operario
# sudo adduser user01
# sudo passwd user01